In [ ]:
# Install Required Libraries
!pip install beautifulsoup4
from bs4 import BeautifulSoup

import requests

In [ ]:
def GetCountriesListDict():
  # Import HTML From Wikipedia
  continents = ["Africa", "Asia","North America", "South America", "Europe", "Australia"]
  HTML_List = {}
  for continent in continents:
    response = requests.get("https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_in_"+continent)
    soup = BeautifulSoup(response.content) # but can also work with response.text string -- using bytes is faster tho
    HTML_List[continent] = soup
  return HTML_List

In [ ]:
# Get list of countries & their wiki links
def ReturnDictOfAllCountriesPerContinent():
  ListOfCountriesPerContinent={}
  for key, value in GetCountriesListDict().items() :
      CountryList = []
      CountryLinkList = []

      if(key=="Europe"):
        TableOfCountries = (value.find_all('table',{"class":"nowraplinks mw-collapsible autocollapse navbox-inner"},{"style":"border-spacing:0;background:transparent;color:inherit"}))
        TableOfCountries = (TableOfCountries[1].find_all('ul'))
        TableOfCountries = (TableOfCountries[1].find_all('li'))
      elif(key=="Asia"):
        TableOfCountries = (value.find('table',{"class":"nowraplinks mw-collapsible autocollapse navbox-inner"},{"style":"border-spacing:0;background:transparent;color:inherit"}))
        TableOfCountries = (TableOfCountries.find_all('ul'))
        TableOfCountries = (TableOfCountries[1].find_all('li'))
      else:
        TableOfCountries = (value.find('td',{"class":"navbox-list navbox-odd"}))
        TableOfCountries = (TableOfCountries.find('ul'))
        TableOfCountries = (TableOfCountries.find_all('li'))

      for Country in (TableOfCountries):
        CountryList.append(Country.find('a')['title'])
        CountryLinkList.append('https://en.wikipedia.org'+Country.find('a')['href'])

      ListOfCountriesPerContinent[key] = [CountryList,CountryLinkList]
  return ListOfCountriesPerContinent

In [ ]:
def ParsePresFromSoup(soup_pres,name):
  BirthDate = []
  OfficeDate = []
  PoliticalParty = []

  for index,label in enumerate(zip(soup_pres.find_all('td',{"class":"infobox-full-data"}))):
    if(label[0].text.find('1') != -1 and label[0].text.find('2') != -1):
      OfficeDate.append(label[0].text[label[0].text.find("office")+6:])
    
  for index,label in enumerate(zip(soup_pres.find_all('th',{"class":"infobox-label"}),soup_pres.find_all('td',{"class":"infobox-data"}))):
    if((label[0].text.find("Born") != -1)):
      #BirthDate.append(label[1].text.split('(')[1].split(')')[0])
      BirthDate.append(label[1].text)


    if((label[0].text.find("party") != -1)):
      PoliticalParty.append(label[1].text.split('[')[0])


  return [[name],BirthDate,OfficeDate,PoliticalParty]

In [ ]:
def ParseCityFromSoup(soup_city,name):
  population = []
  area = []
  Governer = []
  Coords = []

  Coords.append(soup_city.body.find('span',{"class":"latitude"}).text)
  Coords.append(soup_city.body.find('span',{"class":"longitude"}).text)

  for index,label in enumerate(zip(soup_city.find_all('th',{"class":"infobox-label"}),soup_city.find_all('td',{"class":"infobox-data"}))):
    if(label[0].text.find("Total") != -1 or label[0].text.find("estimate") != -1 or label[0].text.find("Metro") != -1 or label[0].text.find("Megacity") != -1 or label[0].text.find("Municipality")):
      x = (label[1]).text.split('[')
      x = x[0].split('(')[0].split('\xa0')
      if(len(x)>1 and (x[1] == "km2" or x[1] == "km2 ")):
        area.append(float(x[0].replace(',','')))
      else:
        if(x[0].find(',') != -1 and x[0].find('m') == -1):
          population.append(x[0].replace(',',''))
      
    if((label[0].text.find("Governor") != -1 or label[0].text.find("Mayor") != -1)and label[0].text.find("Governorate") == -1):
      Governer.append(label[1].text.split('[')[0])

  return [[name],population,area,Governer,Coords]

In [ ]:
def ParseCountryFromSoup(soup,continent,name):
  InfoBox1 = soup.find_all('th',{"class":"infobox-label"})
  InfoBox2 = soup.find_all('td',{"class":"infobox-data"})

  languages = []
  population = []
  drivingSide = []
  CallingCode = []
  Timezone = []
  Area = []
  Water = []
  Legislature = []
  HDI = []
  Currency = []
  GDP_PPP = []
  GDP_Nominal = []
  Gini = []
  CapitalPol = []
  CapitalPol_link = []
  CapitalGeo = []
  CapitalGeo_link = []

  Presidents = []
  CityDetails = []
  PresDetails = []

  is_PPP = True
  for index,label in enumerate(zip(InfoBox1,InfoBox2)):
    x = label[0].text
    if(label[0].text.find("Official") != -1):
      new_list = (label[1].text).split('\n')
      for i in new_list:
        if(i != ''):
          i = i.split('[')
          languages.append(i[0])

    if(label[0].text.find("2020 estimate") != -1 or label[0].text.find("2021 estimate") != -1):
      x = (label[1]).text.split('[')
      x[0] = x[0].split('(')[0]
      population.append(x[0].replace(',',''))
      #population.append(int(x[0].replace(',','')))
      
    #print(label[0].text)
    if(label[0].text.find("Driving") != -1):
      x = label[1].text.split('[')
      drivingSide.append(x[0])

    if(label[0].text.find("Calling") != -1):
      x = label[1].text.split('[')
      CallingCode.append(x[0])

    if(label[0].text.find("Time zone") != -1):
      Timezone.append(label[1].text) 

    if(label[0].text.find("Total") != -1):
      x = label[1].text.split('\xa0')
      if(len(x)>1):
        if(x[1].split(' ')[0] == "km2"):
          x[0] = x[0].split('[')[0]
          #Area.append(int(x[0].replace(',','')))
          Area.append(x[0].replace(',',''))

        else:
          if(is_PPP):
            GDP_PPP.append(x[0]+" "+x[1].split('[')[0])
            is_PPP=False
          else:
            GDP_Nominal.append(x[0]+" "+x[1].split('[')[0])
      else:
        if(is_PPP):
          x = x[0].lstrip().split(' ')
          if(len(x)>1):
            GDP_PPP.append(x[0]+" "+x[1].split('[')[0])
          else:
            GDP_PPP.append(x[0].split('[')[0])

          is_PPP=False
        else:
          x = x[0].lstrip().split(' ')
          GDP_Nominal.append(x[0]+" "+x[1].split('[')[0])

    if(label[0].text.find("Water") != -1):
      if(label[1].text=="negligible"):
        Water.append(0)
      else:
        x = label[1].text.split('(')
        #Water.append(float(x[0]))
        Water.append(x[0])

    if(label[0].text.find("Legislature") != -1):
      Legislature.append(label[1].text)

    if(label[0].text.find("HDI") != -1):
      x = label[1].text.split('\xa0')
      x = x[1].split('[')
      HDI.append(x[0])
      #HDI.append(float(x[0]))

    if(label[0].text.find("Currency") != -1):
      Currency.append(label[1].text.replace('\xa0',' '))

    if(label[0].text.find("Gini") != -1):
      x = label[1].text.split('\xa0')
      if(len(x)>1):
        x = x[1].split('[')
      x = x[0].split('h')
      x = x[0].split('l')

      #Gini.append(float(x[0]))
      Gini.append(x[0])

    if(label[0].text.find("Capital") != -1):
      if(label[0].find('div') and label[0].find('div').text == "and largest city"):
        #print(name)
        if(name=="Bosnia and Herzegovina"):
          temp = label[1].find_all('a')[1]
          CapitalPol.append(temp.text)
          CapitalGeo.append(temp.text)
          CapitalGeo_link.append(temp['href'])
          CapitalPol_link.append(temp['href'])
        else:
          temp = label[1].find('a')
          CapitalPol.append(temp.text)
          CapitalGeo.append(temp.text)
          CapitalGeo_link.append(temp['href'])
          CapitalPol_link.append(temp['href'])
      else:
        if(name=="Monaco"):
          CapitalPol.append("Monte Carlo")
          CapitalPol_link.append("/wiki/Monte Carlo")
        elif(name=="Switzerland"):
          CapitalPol.append("Zürich")
          CapitalPol_link.append("/wiki/Zürich")        
        
        else:
          CapitalPol.append(label[1].find('a').text)
          CapitalPol_link.append(label[1].find('a')['href'])


    if(label[0].text.find("Largest") != -1 ):
      if(label[1].find('a')):
        CapitalGeo_link.append(label[1].find('a')['href'])
        CapitalGeo.append(label[1].find('a').text)
      
    if((label[0].text.find("President") != -1 or label[0].text.find("Monarch") != -1 )and label[0].text.find("Vice") == -1):
      Presidents.append(label[1].text)
      link_pres = label[1].find('a')
      if(link_pres):
        response = requests.get("https://en.wikipedia.org"+link_pres['href'])
        PresDetails.append(ParsePresFromSoup(BeautifulSoup(response.content),label[1].text))

  if(CapitalPol == CapitalGeo and len(CapitalGeo) > 0):
    response = requests.get("https://en.wikipedia.org"+CapitalPol_link[0])
    CityDetails.append(ParseCityFromSoup(BeautifulSoup(response.content),CapitalPol[0]))
  else:
    for link,city in zip(CapitalPol_link+CapitalGeo_link,CapitalPol+CapitalGeo):
      response = requests.get("https://en.wikipedia.org"+link)
      CityDetails.append(ParseCityFromSoup(BeautifulSoup(response.content),city))

  return [[name],[continent],languages,population,drivingSide,CallingCode,Timezone,Area,Water,Legislature,HDI,Currency,GDP_PPP,GDP_Nominal,Gini,CapitalPol,CapitalGeo,Presidents,CityDetails,PresDetails]

In [ ]:
# ----- MAIN -------
Database = {}
ListOfCountriesPerContinent = ReturnDictOfAllCountriesPerContinent()
for continent, value in ListOfCountriesPerContinent.items():
  for name,link in zip(value[0],value[1]):
    print(continent,name,link)
    response = requests.get(link)
    soup = BeautifulSoup(response.content)
    Database[name] = ParseCountryFromSoup(soup,continent,name)



Africa Algeria https://en.wikipedia.org/wiki/Algeria
Africa Angola https://en.wikipedia.org/wiki/Angola
Africa Benin https://en.wikipedia.org/wiki/Benin
Africa Botswana https://en.wikipedia.org/wiki/Botswana
Africa Burkina Faso https://en.wikipedia.org/wiki/Burkina_Faso
Africa Burundi https://en.wikipedia.org/wiki/Burundi
Africa Cameroon https://en.wikipedia.org/wiki/Cameroon
Africa Cape Verde https://en.wikipedia.org/wiki/Cape_Verde
Africa Central African Republic https://en.wikipedia.org/wiki/Central_African_Republic
Africa Chad https://en.wikipedia.org/wiki/Chad
Africa Comoros https://en.wikipedia.org/wiki/Comoros
Africa Democratic Republic of the Congo https://en.wikipedia.org/wiki/Democratic_Republic_of_the_Congo
Africa Republic of the Congo https://en.wikipedia.org/wiki/Republic_of_the_Congo
Africa Djibouti https://en.wikipedia.org/wiki/Djibouti
Africa Egypt https://en.wikipedia.org/wiki/Egypt
Africa Equatorial Guinea https://en.wikipedia.org/wiki/Equatorial_Guinea
Africa Eritrea

In [ ]:
import re
from datetime import datetime

PRESIDENTS_TABLE = []
CITIES_TABLE = []
COUNTRIES_TABLE = []
LANGUAGES_TABLE = []

for row in Database.items():
  continent = row[1][1][0]
  city = []
  languages=[]
  Name = row[1][0]

  if(len(row[1][2])>0):
    for l in row[1][2]:
      languages.append(re.findall('[A-Z][^A-Z]*', l))

    languages = languages[0]
  else:
    languages = [""]

  if(len(row[1][3])>0):
    population = [int(''.join(filter(str.isdigit, row[1][3][0])))]

  else:
    population = [0]

  if(len(row[1][4])>0):
    drivingSide = row[1][4]
    if(drivingSide[0].find("right") != -1):
      drivingSide[0]=1
    elif(drivingSide[0].find("left") != -1):
      drivingSide[0]=0

  else:
    drivingSide = [""]

  if(len(row[1][5])>0):
    if(len(row[1][5][0])<5):
      CallingCode = row[1][5]
    else:
      CallingCode = [""]
  else:
    CallingCode = [""]
 
  if(len(row[1][6])>0):
    Timezone = row[1][6]
  else:
    Timezone = [""]

  if(len(row[1][7])>0):
    if(len(row[1][7])>1):
      Area = [row[1][7][0]]
    else:
      Area = row[1][7]
  else:
    Area = [""]

  if(len(row[1][8])>0):
    if(row[1][8][0]!=0):
      Water = [row[1][8][0].split('[')[0]]
      if(len(Water[0])>8):
        Water = [""]
      Water = [i.replace('%', '') for i in Water]

  else:
    Water = [""]

  if(len(row[1][9])>0):
    if(len(row[1][9][0])>40):
      Legislature = [""]
    else:
      Legislature = row[1][9]
  else:
    Legislature = [""]

  if(len(row[1][10])>0):
    HDI = [row[1][10][0][0:5]]
  else:
    HDI = [""]

  if(len(row[1][11])>0):
    Currency = [row[1][11][0]]
    if(len(Currency[0])>40):
      Currency = [""]
  else:
    Currency = [""]

  if(len(row[1][12])>0):
    GDP_PPP = [row[1][12][0]]
    
    GDP_PPP = GDP_PPP[0].replace('\t',' ').split(' ')

    if(len(GDP_PPP)>1):
      if(GDP_PPP[1] == "million"):
        GDP_PPP[0] = float(GDP_PPP[0].replace('$','').replace(',',''))*1000000
      elif(GDP_PPP[1] == "billion"):
        GDP_PPP[0] = float(GDP_PPP[0].replace('$','').replace('US','').replace(',',''))*1000000000
      elif(GDP_PPP[1] == "trillion"):
        GDP_PPP[0] = float(GDP_PPP[0].replace('$','').replace(',',''))*1000000000000
    else:
      try:
        GDP_PPP[0] = float(GDP_PPP[0].replace('$','').replace('US','').replace(',',''))
      except:
        print('')
  else:
    GDP_PPP = [""]

  
  if(len(row[1][13])>0):
    GDP_Nominal = [row[1][13][0]]
    
    GDP_Nominal = GDP_Nominal[0].replace('\t',' ').split(' ')

    if(len(GDP_Nominal)>1):
      if(GDP_Nominal[1] == "million"):
        GDP_Nominal[0] = float(GDP_Nominal[0].replace('$','').replace(',',''))*1000000
      elif(GDP_Nominal[1] == "billion"):
        try:
          GDP_Nominal[0] = float(GDP_Nominal[0].replace('$','').replace('US','').replace(',',''))*1000000000
        except:
          GDP_Nominal[0] = float(GDP_Nominal[1].replace('$','').replace('US','').replace(',',''))*1000000000
      elif(GDP_Nominal[1] == "trillion"):
        GDP_Nominal[0] = float(GDP_Nominal[0].replace('$','').replace(',',''))*1000000000000
    else:
      try:
        GDP_Nominal[0] = float(GDP_Nominal[0].replace('$','').replace('US','').replace(',',''))
      except:
        print('')
  else:
    GDP_Nominal = [""]

  if(len(row[1][14])>0 and len(row[1][14][0])>2):
    Gini = [float(row[1][14][0][0:4].split("[")[0])]
  else:
    Gini = [""]

  if(len(row[1][15])>0):
    CapitalPol = row[1][15]
  else:
    CapitalPol = [""]

  if(len(row[1][16])>0):
    CapitalGeo = row[1][16]
  else:
    CapitalGeo = [""]
  
  if(len(row[1][17])>0):
    Presidents = [row[1][17][0].split('(')[0].split('[')[0]]
  else:
    Presidents = [""]

  if(len(row[1][18])>0):
    for i in row[1][18]:
      if(len(i[0])>0):
        CityName = i[0]
      else:
        CityName = [""]

      if(len(i[1])>0):
        CityPopulation = i[1]
        if(len(CityPopulation)>1 and CityPopulation[0].find(' ') == -1):
          CityPopulation = [float(CityPopulation[0])]
        else:
          CityPopulation = [""]
      else:
        CityPopulation = [""]

      if(len(i[2])>0):
        CityArea = i[2]
        if(len(CityArea)>1):
          CityArea = [float(CityArea[0])]
      else:
        CityArea = [""]

      if(len(i[3])>0):
        CityGoverner = i[3]
      else:
        CityGoverner = [""]

      if(len(i[4])>0):
        CityCoords = i[4]
      else:
        CityCoords = [""]
      
      if(len(row[1][19])>=0):
        president = (row[1][19])
      else:
        president = ([""])
        

      city.append([CityName,CityPopulation,CityArea,CityGoverner,CityCoords])
  else:
      city.append(["None"])

  if(len(president)>1):
    president = [president[0]]


  for i in president:
    PresName = i[0]
    if(len(i[1])==0):
      PresBirthDate = [""]
    else:
      PresBirthDate = re.findall(r'\(.*?\)', i[1][0])
      if(len(PresBirthDate)==0):
        PresBirthDate = [""]
      elif(len(PresBirthDate)>1):
        PresBirthDate = [PresBirthDate[0][1:-1]]
    
      for index,j in enumerate(PresBirthDate):
        if(j.find('age') != -1):
          PresBirthDate[index] = ""
        elif(j!=""):
            PresBirthDate = [datetime.strptime(PresBirthDate[0], '%Y-%m-%d')]
    
    if(len(i[2])>=1):
      PresOfficeDate = [i[2][0].replace('\xa0',' ').split('–')[0].split('(')[0].split('[')[0].split('Ac')[0]]
      try:
          PresOfficeDate[0] = datetime.strptime(PresOfficeDate[0].lstrip().rstrip(), '%d %B %Y')
      except:
        try:
            PresOfficeDate[0] = datetime.strptime(PresOfficeDate[0].lstrip().rstrip(), '%B %Y')
        except:
            PresOfficeDate[0] = datetime.strptime(PresOfficeDate[0].lstrip().rstrip(), '%B %d, %Y')


    else:
      PresOfficeDate=[""]

    if(len(i[3])>=1):
      PresPoliticalParty = [i[3][0].split('(')[0]]
    else:
      PresPoliticalParty=[""]

    PRESIDENTS_TABLE.append([len(PRESIDENTS_TABLE)+1,PresName[0].split('[')[0].split('(')[0], PresBirthDate[0], PresOfficeDate[0], PresPoliticalParty[0]])

  for i in city:
    if(i != ['None']):
      CITIES_TABLE.append([len(CITIES_TABLE)+1,i[0][0],i[1][0],i[2][0],i[3][0],[i[4][0]+" "+i[4][1]][0]])  # CREATE MYSQL FOR CITY
  for index,row in enumerate(CITIES_TABLE):
    if(row[1] == CapitalPol[0]):
      CapitalPol[0] = index+1

    if(row[1] == CapitalGeo[0]):
      CapitalGeo[0] = index+1

  for index,row in enumerate(PRESIDENTS_TABLE):
    if(row[1] == Presidents[0]):
      Presidents[0] = index+1

  for i in languages:
    LANGUAGES_TABLE.append([len(COUNTRIES_TABLE)+1,i])
  COUNTRIES_TABLE.append([len(COUNTRIES_TABLE)+1,Name[0],continent,population[0],drivingSide[0],CallingCode[0],Timezone[0],Area[0],Water[0],Legislature[0],HDI[0],Currency[0],GDP_PPP[0],GDP_Nominal[0],Gini[0],CapitalPol[0],CapitalGeo[0],Presidents[0]])
    
  for index,row in enumerate(COUNTRIES_TABLE):
    try:
      COUNTRIES_TABLE[index][-1] = int(row[-1])
    except:
      COUNTRIES_TABLE[index][-1] = ''

In [ ]:
# Get COVID Cases
import csv
import requests

CSV_URL = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'

dictionary={}
with requests.Session() as s:
    download = s.get(CSV_URL)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

    for row in (my_list):
      if(row[4] != '' and row[34] != ''):
        dictionary[row[2]] = [row[3],row[4],row[34]]

for i,j in dictionary.items():
  for index,row in enumerate(COUNTRIES_TABLE):
    if(row[1] == i):
      COUNTRIES_TABLE[index].append(j[1])
      COUNTRIES_TABLE[index].append(j[2])


In [ ]:
import csv
with open('Presidents.csv', mode='w') as csv_file:
  csv_file = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  csv_file.writerow(["id","Name","BirthDate","OfficeDate","PoliticalParty"])
  for i in PRESIDENTS_TABLE:
    csv_file.writerow(i)

with open('City.csv', mode='w') as csv_file:
  csv_file = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  csv_file.writerow(["id","Name","Population","Area","Governer","Coordinates"])
  for i in CITIES_TABLE:
    csv_file.writerow(i)

with open('Country.csv', mode='w') as csv_file:
  csv_file = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  csv_file.writerow(["id","Name","Continent","Population","isDrivingRightSide","CallingCode","Timezone","Area","Water","Legislature","HDI","Currency","GDP_PPP","GDP_Nominal","Gini","PolCapital","GeoCapital","Presidents","TotalCovidCases","TotalCovidVaccinations"])
  for i in COUNTRIES_TABLE:
    csv_file.writerow(i)

with open('Official Languages.csv', mode='w') as csv_file:
  csv_file = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  csv_file.writerow(["CountryID","Language"])
  for i in LANGUAGES_TABLE:
    csv_file.writerow(i)